In [ ]:
import pandas as pd    ###

# 엑셀 파일 경로를 지정
xlsx_file = pd.read_excel(r'C:\temp\online.xlsx') ##  online_retail_ii을 online.csv로 바꿈...

# csv로 변환한 후 파일명과 저장 경로 지정
xlsx_file.to_csv(r'C:\temp\online.csv', index=None, header=True )

In [ ]:
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from scipy import stats as stats

In [ ]:
df=pd.read_csv('c:/temp/online.csv')

In [ ]:
df

In [ ]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate']).dt.strftime('%Y-%m-%d')


In [ ]:

df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

df = df.dropna(subset=['Customer ID'])

In [ ]:
# 1. Customer ID를 정수형으로 변환 
df['Customer ID'] = df['Customer ID'].astype(int)

# 2. InvoiceDate를 기준으로 정렬 (오름차순)
df = df.sort_values(by='InvoiceDate')

# 내림차순 정렬을 원할 경우:
# df = df.sort_values(by='InvoiceDate', ascending=False)


In [ ]:
# 1. Total 컬럼 생성 (Quantity * Price)
df['Total'] = df['Quantity'] * df['Price']

# 결과 확인
print(df[['Quantity', 'Price', 'Total']].head())

In [ ]:
# 1. 국가별 고유 고객 수 계산
unique_customers = df.groupby('Country')['Customer ID'].nunique().reset_index()
unique_customers.columns = ['Country', 'Unique Customers']  # 컬럼명 변경

# 2. 시각화 (막대 그래프)
plt.figure(figsize=(12, 6))
sns.barplot(
    x='Country', 
    y='Unique Customers', 
    data=unique_customers.sort_values('Unique Customers', ascending=False),
    palette='viridis'
)

# 그래프 제목 및 레이블 설정
plt.title('Number of Unique Customers by Country', fontsize=15)
plt.xlabel('Country', fontsize=12)
plt.ylabel('Number of Unique Customers', fontsize=12)
plt.xticks(rotation=45)  # 국가명이 길 경우 회전

# 값 표시 (각 막대 위에 숫자 표시)
for index, value in enumerate(unique_customers.sort_values('Unique Customers', ascending=False)['Unique Customers']):
    plt.text(index, value, str(value), ha='center', va='bottom')

plt.tight_layout()
plt.show()

In [ ]:
# 1. 각 고객의 첫 구매 날짜 계산
first_purchase = df.groupby('Customer ID')['InvoiceDate'].min().reset_index()
first_purchase.columns = ['Customer ID', 'FirstPurchaseDate']  # 컬럼명 변경

# 2. 원본 데이터에 첫 구매 날짜 병합
df = pd.merge(df, first_purchase, on='Customer ID')

# 3. 첫 구매 날짜에서 '년-월' 추출
df['FirstPurchaseMonth'] = df['FirstPurchaseDate'].dt.to_period('M').astype(str)


In [ ]:
# 월별 신규 고객 수 집계
monthly_new_customers = df.groupby('FirstPurchaseMonth')['Customer ID'].nunique().reset_index()
monthly_new_customers.columns = ['Month', 'NewCustomers']  # 컬럼명 변경

In [ ]:
plt.figure(figsize=(14, 7))

# 막대 그래프 (신규 고객 수)
sns.barplot(
    x='Month', 
    y='NewCustomers', 
    data=monthly_new_customers,
    color='skyblue',
    alpha=0.6
)

# 라인 플롯 (트렌드 강조)
sns.lineplot(
    x='Month', 
    y='NewCustomers', 
    data=monthly_new_customers,
    color='red', 
    marker='o',
    linewidth=2,
    markersize=8,
    label='Trend'
)

# 그래프 스타일링
plt.title('Monthly New Customer Acquisition', fontsize=16, pad=20)
plt.xlabel('Month', fontsize=12)
plt.ylabel('Number of New Customers', fontsize=12)
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.4)

# 값 표시 (막대 위에 숫자 표시)
for index, row in monthly_new_customers.iterrows():
    plt.text(
        index, 
        row['NewCustomers'] + 0.5,  # Y축 위치 조정
        str(row['NewCustomers']), 
        ha='center', 
        va='bottom',
        fontsize=10
    )

plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# 필수 컬럼 확인: Customer ID, InvoiceDate, Total(구매 금액)
print(df[["Customer ID", "InvoiceDate", "Total"]].head())


In [ ]:
# 1. 각 고객의 첫 구매 날짜 추출
first_purchase = df.groupby("Customer ID")["InvoiceDate"].min().reset_index()
first_purchase.columns = ["Customer ID", "FirstPurchaseDate"]

In [ ]:
# 2. 첫 구매 이후의 모든 거래에서 총 구매 금액 계산
def calculate_total_spent_after_first_purchase(row):
    customer_id = row["Customer ID"]
    first_date = row["FirstPurchaseDate"]
    total_spent = df[(df["Customer ID"] == customer_id) & 
                     (df["InvoiceDate"] >= first_date)]["Total"].sum()
    return total_spent

first_purchase["TotalSpentAfterFirstPurchase"] = first_purchase.apply(
    calculate_total_spent_after_first_purchase, axis=1
)

In [ ]:
# 활성화 기준 (£50 이상)
ACTIVATION_THRESHOLD = 50

# 1. 전체 고객 수
total_customers = len(first_purchase)

# 2. 활성화된 고객 수 (첫 구매 후 £50 이상 구매)
activated_customers = first_purchase[first_purchase["TotalSpentAfterFirstPurchase"] >= ACTIVATION_THRESHOLD]
activated_count = len(activated_customers)

# 3. 활성화율 계산
activation_rate = (activated_count / total_customers) * 100

# 결과 출력
print(f"전체 고객 수: {total_customers}")
print(f"활성화된 고객 수 (£{ACTIVATION_THRESHOLD}+): {activated_count}")
print(f"활성화율: {activation_rate:.2f}%")


In [ ]:
plt.figure(figsize=(10, 6))
data = {
    "Category": ["Total Customers", "Activated Customers"],
    "Count": [total_customers, activated_count]
}
df_viz = pd.DataFrame(data)

# 막대 그래프
ax = sns.barplot(x="Category", y="Count", data=df_viz, palette=["skyblue", "salmon"])

# 값 및 비율 표시
for p in ax.patches:
    height = p.get_height()
    ax.text(p.get_x() + p.get_width()/2., height + 5,
            f"{height:,}\n({height/total_customers*100:.1f}%)",
            ha="center", fontsize=12)

plt.title("Customer Activation Analysis", fontsize=16)
plt.ylabel("Number of Customers", fontsize=12)
plt.xticks(fontsize=12)
sns.despine()
plt.show()

In [ ]:
# 1. 각 고객의 첫 구매 분기(코호트) 계산
df["InvoiceQuarter"] = df["InvoiceDate"].dt.to_period("Q")
df["CohortQuarter"] = df.groupby("Customer ID")["InvoiceDate"].transform("min").dt.to_period("Q")

# 2. 코호트-분기별 고객 활동 집계
cohort_data = df.groupby(["CohortQuarter", "InvoiceQuarter"]).agg(
    ActiveCustomers=("Customer ID", "nunique"),  # 활성 고객 수
    TotalRevenue=("Total", "sum")              # 매출액 (옵션)
).reset_index()

# 3. 코호트 인덱스(분기 간격) 계산
cohort_data["Periods"] = (cohort_data["InvoiceQuarter"] - cohort_data["CohortQuarter"]).apply(
    lambda x: x.n if x.n >= 0 else 0  # 음수 제거
)

In [ ]:
##고객 유지율 계산

# 1. 각 코호트의 초기 고객 수 추출
cohort_size = cohort_data[cohort_data["Periods"] == 0][["CohortQuarter", "ActiveCustomers"]]
cohort_size.columns = ["CohortQuarter", "InitialCustomers"]

# 2. 유지율 계산 (%)
retention = pd.merge(cohort_data, cohort_size, on="CohortQuarter")
retention["RetentionRate"] = (retention["ActiveCustomers"] / retention["InitialCustomers"]) * 100

# 3. 피벗 테이블 생성 (시각화용)
retention_pivot = retention.pivot_table(
    index="CohortQuarter",
    columns="Periods",
    values="RetentionRate",
    aggfunc="mean"
)

In [ ]:
plt.figure(figsize=(12, 8))
sns.heatmap(
    retention_pivot,
    annot=True,
    fmt=".2f",
    cmap="Greens",
    cbar_kws={'label':'Retention Rate, %'},
    linewidths=0.5,
    linecolor='gray',
    mask=retention_pivot.isnull()
)
plt.title("Quarterly Cohort Retention Rate (%)", fontsize=16)
plt.xlabel("Quarters After First Purchase", fontsize=12)
plt.ylabel("Cohort Quarter", fontsize=12)
plt.show()

In [ ]:
# 1. 각 고객의 첫 구매 분기(코호트)와 거래 분기 계산
df["InvoiceQuarter"] = df["InvoiceDate"].dt.to_period("Q")
df["CohortQuarter"] = df.groupby("Customer ID")["InvoiceDate"].transform("min").dt.to_period("Q")

# 2. 코호트-분기별 평균 구매수량 집계
cohort_qty = df.groupby(["CohortQuarter", "InvoiceQuarter"]).agg(
    AvgQuantity=("Quantity", "mean"),  # 평균 구매수량
    ActiveCustomers=("Customer ID", "nunique")  # 활성 고객 수 (옵션)
).reset_index()

# 3. 코호트 인덱스(분기 간격) 계산
cohort_qty["Periods"] = (cohort_qty["InvoiceQuarter"] - cohort_qty["CohortQuarter"]).apply(
    lambda x: x.n if x.n >= 0 else 0
)

In [ ]:
# 평균 구매수량 피벗 테이블
qty_pivot = cohort_qty.pivot_table(
    index="CohortQuarter",
    columns="Periods",
    values="AvgQuantity",
    aggfunc="mean"
)

# 활성 고객 수 피벗 테이블 (옵션)
customer_pivot = cohort_qty.pivot_table(
    index="CohortQuarter",
    columns="Periods",
    values="ActiveCustomers",
    aggfunc="sum"
)

In [ ]:
plt.figure(figsize=(12, 8))
sns.heatmap(
    qty_pivot,
    annot=True,
    fmt=".1f",  # 소수점 1자리 표시
    cmap="Blues",
    linewidths=0.5,
    mask=qty_pivot.isnull(),
    cbar_kws={"label": "Average Quantity"}
)
plt.title("Quarterly Cohort Analysis: Average Purchase Quantity", fontsize=16)
plt.xlabel("Quarters After First Purchase", fontsize=12)
plt.ylabel("Cohort Quarter", fontsize=12)
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
for cohort in qty_pivot.index:
    plt.plot(
        qty_pivot.columns,
        qty_pivot.loc[cohort],
        marker="o",
        label=f"Cohort {cohort}"
    )
plt.title("Average Quantity Trend by Cohort", fontsize=16)
plt.xlabel("Quarters After First Purchase", fontsize=12)
plt.ylabel("Average Quantity", fontsize=12)
plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left")
plt.grid(linestyle="--", alpha=0.4)
plt.tight_layout()
plt.show()

In [ ]:
# 1. 월별 집계 (총 수익 + 활성 고객 수)
monthly_data = df.resample('M', on='InvoiceDate').agg(
    TotalRevenue=('Total', 'sum'),      # 월별 총 수익
    ActiveCustomers=('Customer ID', 'nunique')  # 월별 활성 고객 수
).reset_index()

# 2. ARPU 계산
monthly_data['ARPU'] = monthly_data['TotalRevenue'] / monthly_data['ActiveCustomers']

# 결과 확인
print(monthly_data[['InvoiceDate', 'TotalRevenue', 'ActiveCustomers', 'ARPU']])


In [ ]:
##월별추이

plt.figure(figsize=(14, 6))
plt.plot(
    monthly_data['InvoiceDate'],
    monthly_data['ARPU'],
    marker='o',
    linestyle='-',
    color='royalblue',
    label='ARPU'
)


In [ ]:
plt.xlabel('Month', fontsize=12)
plt.ylabel('ARPU', fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.4)
plt.xticks(rotation=45)

# 최대/최소 ARPU 강조
max_arpu = monthly_data['ARPU'].max()
min_arpu = monthly_data['ARPU'].min()
plt.axhline(max_arpu, color='red', linestyle=':', alpha=0.5, label=f'Max ARPU: {max_arpu:.2f}')
plt.axhline(min_arpu, color='green', linestyle=':', alpha=0.5, label=f'Min ARPU: {min_arpu:.2f}')

plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
fig, ax1 = plt.subplots(figsize=(14, 6))

# ARPU (첫 번째 축)
ax1.plot(
    monthly_data['InvoiceDate'],
    monthly_data['ARPU'],
    color='red',
    marker='o',
    label='ARPU'
)
ax1.set_xlabel('Month')
ax1.set_ylabel('ARPU', color='red')
ax1.tick_params(axis='y', labelcolor='red')

# 활성 고객 수 (두 번째 축)
ax2 = ax1.twinx()
ax2.bar(
    monthly_data['InvoiceDate'],
    monthly_data['ActiveCustomers'],
    color='skyblue',
    alpha=0.4,
    label='Active Customers'
)
ax2.set_ylabel('Active Customers', color='skyblue')
ax2.tick_params(axis='y', labelcolor='skyblue')

plt.title('ARPU vs Active Customers', fontsize=16)
fig.legend(loc='upper right')
plt.show()

In [ ]:
# 파라미터 설정--
low = 5    # 최소 대기시간 (분)
high = 15  # 최대 대기시간 (분)
num_customers = 100  # 고객 수

# 균등분포로 대기시간 생성
waiting_times = np.random.uniform(low=low, high=high, size=num_customers)

# 결과 출력 (첫 10개 값 샘플)
print("시뮬레이션된 대기시간 예시 (첫 10명):")
print(waiting_times[:10].round(2))

In [ ]:
# 통계 계산
mean_waiting = np.mean(waiting_times).round(2)
std_waiting = np.std(waiting_times).round(2)

print(f"\n평균 대기시간: {mean_waiting} 분")
print(f"표준편차: {std_waiting} 분")

In [ ]:
plt.figure(figsize=(10, 6))
plt.hist(waiting_times, bins=15, color='skyblue', edgecolor='black', alpha=0.7)
plt.axvline(mean_waiting, color='red', linestyle='dashed', linewidth=2, label=f'평균: {mean_waiting}분')
plt.title('고객 대기시간 분포 (균등분포)', fontsize=15)
plt.xlabel('대기시간 (분)', fontsize=12)
plt.ylabel('고객 수', fontsize=12)
plt.legend()
plt.grid(axis='y', alpha=0.3)
plt.show()

In [ ]:
# 설정
p_success = 0.3
num_trials = 10

# 각 시도의 성공 여부 (1: 성공, 0: 실패)
results = np.random.binomial(n=1, p=p_success, size=num_trials)

# 총 성공 횟수
total_success = np.sum(results)

# 출력
print("각 시도의 성공 여부:", results)
print("총 성공 횟수:", total_success)

In [ ]:
num_students = 20
num_questions = 5
p_correct = 0.7

# 각 학생의 점수 시뮬레이션 (5문제에 대해 Bernoulli 시행 후 합계)
scores = np.random.binomial(n=num_questions, p=p_correct, size=num_students)

# 평균 점수 계산
average_score = np.mean(scores)

print("각 학생의 점수:", scores)
print("전체 평균 점수:", average_score)

In [ ]:
# 설정
mu = 50      # 평균
sigma = 5    # 표준편차
sample_size = 1000

# 정규분포에서 1000개 샘플 생성
weights = np.random.normal(mu, sigma, sample_size)

# 45g 이상 55g 이하인 제품의 비율 계산
within_range = np.sum((weights >= 45) & (weights <= 55)) / sample_size
print(f"45g 이상 55g 이하인 제품의 비율: {within_range:.2%}")

# 히스토그램 그리기
plt.hist(weights, bins=30, edgecolor='black', alpha=0.7)
plt.axvline(x=45, color='red', linestyle='--', label='45g')
plt.axvline(x=55, color='red', linestyle='--', label='55g')
plt.title('제품 무게 분포 (정규분포)')
plt.xlabel('무게 (g)')
plt.ylabel('빈도')
plt.legend()
plt.grid(True)